## Creating Naive Bayes Classifier using only NumPy and Pandas

In [15]:
import numpy as np
import pandas as pd

In [16]:
def filter_col(df):
    _df = pd.read_csv(df)
    return _df.drop("Unnamed: 0", axis = 1)

In [17]:
def add_genre(df, genre):
    df['genre'] = genre
    return df

In [18]:
lyrics_df = pd.concat([
    add_genre(pd.read_csv("lyrics_datasets\ArianaGrande.csv"), 'pop'),
    add_genre(filter_col("lyrics_datasets\CardiB.csv"), 'rap'),
    add_genre(filter_col("lyrics_datasets\EdSheeran.csv"), 'pop'),
    add_genre(filter_col("lyrics_datasets\Eminem.csv"), 'rap'),
    add_genre(filter_col(r"lyrics_datasets\NickiMinaj.csv"), 'rap'),
    add_genre(filter_col("lyrics_datasets\Drake.csv"), 'rap'),
    add_genre(filter_col("lyrics_datasets\TaylorSwift.csv"), 'pop'),
    add_genre(filter_col("lyrics_datasets\KatyPerry.csv"), 'pop')
], axis = 0)

In [19]:
lyrics_df.drop(['Artist', 'Title', 'Date', 'Year', 'Album'], axis = 1, inplace=True)

In [20]:
lyrics_df.head()

,Lyric,genre
0,thought i'd end up with sean but he wasn't a m...,pop
1,yeah breakfast at tiffany's and bottles of bub...,pop
2,you you love it how i move you you love it how...,pop
3,ariana grande nicki minaj i've been here all ...,pop
4,right now i'm in a state of mind i wanna be in...,pop


In [21]:
lyrics_df['genre'].value_counts()

pop    1408
rap    1385
Name: genre, dtype: int64

In [94]:
genre_dict = {'pop': 1, 'rap': 0}
lyrics_df['genre'] = lyrics_df['genre'].map(genre_dict)

In [22]:
lyrics_df.isna().sum()

Lyric    9
genre    0
dtype: int64

In [23]:
lyrics_df.dropna(axis = 0, inplace=True)